In [ ]:
class HashTableFullError(Exception):
    pass

class BaseHashTable:
    """
    Базовый класс для хеш-таблиц.
    Отвечает за вычисление хеша и хранение общих параметров (размер, емкость).
    Сама структура хранения (buckets) будет определяться в наследниках,
    так как она зависит от метода разрешения коллизий.
    """
    def __init__(self, capacity: int):
        self.capacity = capacity # Максимальное (или начальное) кол-во слотов
        self.size = 0            # Текущее количество элементов
        # Саму таблицу self.table здесь не создаем, 
        # потому что в одном случае это список списков, в другом — просто список.

    def _hash(self, key):
        """
        Простая хеш-функция: остаток от деления.
        Эту логику мы пишем один раз здесь и используем во всех наследниках.
        """
        # Используем встроенный hash(), чтобы можно было класть строки и числа
        return hash(key) % self.capacity

    def put(self, key, value):
        """
        Метод вставки. Должен быть реализован в наследниках.
        """
        raise NotImplementedError("Метод put должен быть реализован в подклассе")

    def get(self, key):
        """
        Метод поиска. Должен быть реализован в наследниках.
        """
        raise NotImplementedError("Метод get должен быть реализован в подклассе")

    def __len__(self):
        return self.size


class ChainingHashTable(BaseHashTable):
    """
    Хеш-таблица с разрешением коллизий методом цепочек (Chaining).
    В каждой ячейке массива лежит список (bucket).
    Если случается коллизия, мы просто добавляем элемент в конец списка.
    """
    def __init__(self, capacity=10):
        # 1. Не забудьте инициализировать родителя (установить self.capacity)
        # TODO: call super init
        
        # 2. Создаем таблицу. Это должен быть список списков. 
        # [[], [], [], ...]
        self.table = [[] for _ in range(capacity)]

    def put(self, key, value):
        # 1. Вычисляем индекс с помощью родительского метода _hash
        index = self._hash(key)
        
        # 2. Проверяем bucket по этому индексу.
        # Если такой ключ уже есть — обновляем значение.
        bucket = self.table[index]
        
        for i, (k, v) in enumerate(bucket):
            if k == key:
                # TODO: обновить значение tuple или перезаписать элемент
                # Подсказка: кортежи в питоне неизменяемые, проще заменить элемент списка
                bucket[i] = (key, value)
                return

        # 3. Если ключа нет, добавляем новую пару (key, value) в конец списка
        # TODO: добавить в bucket и увеличить self.size
        pass

    def get(self, key):
        # 1. Вычисляем индекс
        index = self._hash(key)
        
        # 2. Ищем ключ в соответствующем bucket'е
        bucket = self.table[index]
        
        # TODO: Пробежаться по bucket, найти ключ и вернуть value. 
        # Если не нашли — вернуть None (или кинуть ошибку, на ваш выбор)
        pass


class LinearProbingHashTable(BaseHashTable):
    """
    Хеш-таблица с открытой адресацией (Linear Probing).
    Если ячейка занята, мы берем следующую (index + 1), пока не найдем пустую.
    """
    def __init__(self, capacity=10):
        # TODO: Вызвать конструктор родителя
        
        # В этом случае таблица — это просто список фиксированной длины.
        # Изначально он заполнен None.
        self.table = [None] * capacity

    def put(self, key, value):
        if self.size == self.capacity:
            raise HashTableFullError("Таблица переполнена")

        # 1. Получаем стартовый индекс
        start_index = self._hash(key)
        index = start_index

        # 2. Ищем свободное место или существующий ключ
        # Цикл должен продолжаться, пока мы не найдем None или наш ключ
        while self.table[index] is not None:
            current_key, current_val = self.table[index]
            
            if current_key == key:
                # Нашли такой же ключ — обновляем (перезапись)
                # TODO: обновить ячейку self.table[index]
                return
            
            # Идем к следующей ячейке (циклически!)
            # TODO: Сдвинуть index на 1 вперед. Не забудьте про % self.capacity,
            # чтобы не выйти за границы массива.
            index = ... # допишите формулу 
            
            # Защита от бесконечного цикла (на случай ошибок в логике size)
            if index == start_index:
                raise HashTableFullError("Что-то пошло не так, вернулись в начало")

        # 3. Нашли пустую ячейку (None). Записываем туда (key, value)
        # TODO: записать пару, увеличить self.size
        pass

    def get(self, key):
        start_index = self._hash(key)
        index = start_index

        # Ищем ключ по тому же алгоритму, что и при вставке
        while self.table[index] is not None:
            current_key, current_val = self.table[index]
            
            if current_key == key:
                # TODO: Что мы возвращаем?
                pass
            
            # TODO: Сдвигаем индекс дальше
            if index == start_index: # Прошли круг и не нашли
                break
                
        return None


# --- Блок проверки ---
if __name__ == "__main__":
    print("\n--- Тестируем метод цепочек (Chaining) ---")
    try:
        # Создаем маленькую таблицу, чтобы специально вызвать коллизии
        h1 = ChainingHashTable(capacity=5)
        
        # Ключи 5 и 10 дадут одинаковый хеш (0), если capacity=5
        print("Вставка 5 -> 'A'")
        h1.put(5, 'A')
        print("Вставка 10 -> 'B' (должна быть коллизия в бакете 0)")
        h1.put(10, 'B')
        print("Вставка 15 -> 'C' (еще одна коллизия)")
        h1.put(15, 'C') # 'C' перезапишет? Нет, ключи разные 5, 10, 15

        print(f"Значение для 5: {h1.get(5)}")   # Ожидаем A
        print(f"Значение для 10: {h1.get(10)}") # Ожидаем B
        print(f"Значение для 15: {h1.get(15)}") # Ожидаем C
        
        # Проверка внутренней структуры (нечестный доступ, но для отладки полезно)
        print(f"Внутренняя таблица: {h1.table}")
        
    except NotImplementedError:
        print("ОШИБКА: Не реализованы методы в ChainingHashTable")
    except Exception as e:
        print(f"ОШИБКА КОДА: {e}")


    print("\n--- Тестируем линейное пробирование (Linear Probing) ---")
    try:
        h2 = LinearProbingHashTable(capacity=5)
        
        # Те же ключи: 5, 10, 15. Все хотят в индекс 0.
        # 5 встанет в [0].
        # 10 увидит, что [0] занят, и встанет в [1].
        # 15 увидит, что [0] и [1] заняты, и встанет в [2].
        
        print("Вставка 5...")
        h2.put(5, 'A')
        print("Вставка 10...")
        h2.put(10, 'B')
        print("Вставка 15...")
        h2.put(15, 'C')

        print(f"Значение для 5: {h2.get(5)}") 
        print(f"Значение для 10: {h2.get(10)}") 
        print(f"Значение для 15: {h2.get(15)}") 

        # Вывод "сырой" таблицы, чтобы увидеть смещение
        print(f"Сырая таблица: {h2.table}")
        
        # Проверка обновления
        h2.put(10, 'NewB')
        print(f"Обновленный 10: {h2.get(10)}") 

    except NotImplementedError:
        print("ОШИБКА: Не реализованы методы в LinearProbingHashTable")
    except Exception as e:
        print(f"ОШИБКА КОДА: {e}")
